In [ ]:
import h5py
import numpy as np
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
import scipy.stats as stats
import seaborn as sns

#conda install -c conda-forge miktex in command prompt
import seaborn as sns
from ipywidgets import widgets, Layout#this is optionally, must be installed 
from IPython import display #this is optionally
from ipywidgets import widgets, Layout


In [ ]:
#FOR LATEX -takes longer to run plots

plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']# all text in blots bold
#plt.rc('text', usetex=True)# for figure export 
plt.rc('text', usetex=False)# use tex for image export
PLTSCALFACTOR =1.5# change this to scale all plots labels (3 is good for export on 4k screan)
SMALL_SIZE = 12 * PLTSCALFACTOR
MEDIUM_SIZE = 15 * PLTSCALFACTOR
BIGGER_SIZE = 18 * PLTSCALFACTOR

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams['lines.linewidth'] = PLTSCALFACTOR

# 2.Extract the data

Data for ILC comparison is extracted from the HDF5 files separately for PTB and CEM. The extracted data will be sorted by frequency at the end of the Notebook and saved into Excel file.

In [ ]:
def extract_data(filename, sensor_index):
    #explore the HDF5 file, folders and subfolders
    with h5py.File(filename,'r') as f:
        base_items=list(f.items())
        print("\nItems in directory", base_items)
        rawtransfer=f.get("RAWTRANSFERFUNCTION")
        rawtransfer_items=list(rawtransfer.items())
        print("\nItems in reference", rawtransfer_items)
        subgroup=rawtransfer.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250")
        subgroup_items=list(subgroup.items())
        print("\n"+sensor_index+"_MPU_9250 items:",subgroup_items)
        subgroup_acceleration=subgroup.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration")
        subgroup_acceleration_items=list(subgroup_acceleration.items())
        print("\nAcceleration items:",subgroup_acceleration_items)
        subgroup_acceleration_5mem=subgroup.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration")
        subgroup_acceleration_5mem_items=list(subgroup_acceleration_5mem.items())
        print("\nAcceleration items_5members:", subgroup_acceleration_5mem_items)
        frequency=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_frequency")
        frequency_items=list(frequency.items())
        print("\nFrequency", frequency_items)
        magnitude=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Magnitude")
        magnitude_items=list(magnitude.items())
        print("\nMagnitude", magnitude_items)
        phase=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Phase")
        phase_items=list(magnitude.items())
        print("\nPhase", phase_items)
        
        
        #extract frequencies, magnitude, phase, uncertainties and all excitation parameters
        frequency_values=np.array(frequency.get("value"))
        magnitude_values=np.array(magnitude.get("value"))
        magnitude_uncertainties=np.array(magnitude.get("uncertainty"))
        phase_values=np.array(phase.get("value"))
        phase_uncertainties=np.array(phase.get("uncertainty"))
        excitation_freq_items=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_frequency")
        excitation_freq=np.array(excitation_freq_items.get("value"))
        excitation_amp_items=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_amplitude")
        excitation_amp=np.array(excitation_amp_items.get("value"))
        excitation_amp_uncertainty=np.array(excitation_amp_items.get("uncertainty"))
        
        #join all necessary data in 2D array
        total_array=np.stack((frequency_values,magnitude_values,magnitude_uncertainties,phase_values, phase_uncertainties,excitation_freq,excitation_amp,excitation_amp_uncertainty), axis=1)
        print("\nArray dimensions:", total_array.shape)
        column_names=["Frequency in Hz", r"$|S(\omega)|$ in $ \frac{\mathrm{m s}^-2}{\mathrm{m s}^-2}$",r"$U_{|S(\omega)|}$ in $ \frac{\mathrm{m s}^-2}{\mathrm{m s}^-2}$", r"$\varphi(\omega)$ in $rad$", r"$U_{\varphi(\omega)}$ in $rad$","Excitation_freq in Hz",r"$A_{excit}$ in $ \frac{\mathrm{m s}^-2}{\mathrm{m s}^-2}$",r"$U_{Aexcit}$ in $ \frac{\mathrm{m s}^-2}{\mathrm{m s}^-2}$"]
        whole_dataset=pd.DataFrame(total_array, columns=column_names)
        f.close()
        
        
        return whole_dataset

In [ ]:
whole_dataset_PTB = extract_data('MPU9250PTB_v5.hdf5',"0x1fe40000")

In [ ]:
whole_dataset_PTB.head(2)

Phase data for PTB must be reverted:

In [ ]:
whole_dataset_PTB[[r"$\varphi(\omega)$ in $rad$"]] = whole_dataset_PTB[[r"$\varphi(\omega)$ in $rad$"]]

In [ ]:
whole_dataset_PTB.head(2)

In [ ]:
whole_dataset_CEM = extract_data('MPU9250CEM_v5.hdf5',"0xbccb0000")

In [ ]:
whole_dataset_CEM[[r"$\varphi(\omega)$ in $rad$"]] = whole_dataset_CEM[[r"$\varphi(\omega)$ in $rad$"]]-np.pi
whole_dataset_CEM.head(2)

Cycles in CEM's dataset start with 80.0 Hz and 250.0 Hz instead of 10.0 Hz. These starting points are deleted in order to compare the cycles in a range from 10.0 Hz and 250.Hz.

In [ ]:
delete_rows=[]

for k in range(0,171,19):
    i=k
    j=k+1
    delete_rows.append(i)
    delete_rows.append(j)
whole_dataset_CEM_new=whole_dataset_CEM.drop(axis=0,index=delete_rows)

# 3.Data analysis

In [ ]:
def split_data_by_frequencies(dataset):
    dict_of_frequencies=dict(iter(dataset.groupby('Frequency [Hz]')))
    return dict_of_frequencies
    #list_of_frequencies=np.array([10,12.5,16,20,25,31.5,40,46.7,50,53.3,63,80,100,125,160,200,250])

In [ ]:
#check if all frequencies are the same
PTB_separated_by_freq=split_data_by_frequencies(whole_dataset_PTB)
CEM_separated_by_freq=split_data_by_frequencies(whole_dataset_CEM)
CEM_separated_by_freq_new=split_data_by_frequencies(whole_dataset_CEM_new)
print("Frequencies - PTB:",PTB_separated_by_freq.keys())
print("Frequencies - CEM:",CEM_separated_by_freq.keys())

In [ ]:
PTB_separated_by_freq.get(10).head(1)

In [ ]:
CEM_separated_by_freq_new.get(10).head(1)

In [ ]:
q_names=list(PTB_separated_by_freq.get(10).columns)

# 4. ANOVA for experiments performed at a given frequency

Ten measurement cycles for PTB and nine for CEM result with ten (or nine) experiments at each calibration point between 10.0 Hz and 250.0 Hz. Each experiment is described with:  ${x_{M}},{x_{\phi}}$ and assigned expanded uncertainties:
${U_{M}},{U_{\phi}}$. These mean values arise from the sine-fitting and conversion of time-domain signals to the frequency domain.  The ANOVA method is used to examine if the mean values of experiments at a given frequency are equal for each laboratory, PTB and CEM respectively. 

<br>In a general case, one-way ANOVA uses the following null and alternative hypotheses:

<br>H0 (a null hypothesis): μ1 = μ2 = μ3 = … = μk (all the population means are equal)
<br>H1 (a research hypothesis): at least one population mean is different from the rest.

There are three primary assumptions in ANOVA:

<br>*The responses for each factor level have a normal population distribution.*
<br>*These distributions have the same variance.*
<br>*The data are independent.*
<br>Violations to the first two that are not extreme can be considered not serious. The sampling distribution of the test statistic is fairly robust, especially as sample size increases and more so if the sample sizes for all factor levels are equal. If you conduct an ANOVA test, you should always try to keep the same sample sizes for each factor level.

If our samples have unequal variances (heteroscedasticity), on the other hand, it can affect the Type I error rate and lead to false positives. This is, basically, what equality of variances means.

<b>A general rule of thumb for equal variances is to compare the smallest and largest sample standard deviations. This is much like the rule of thumb for equal variances for the test for independent means. If the ratio of these two sample standard deviations falls within 0.5 to 2, then it may be that the assumption is not violated.<b>

## 4.1 Test for the equality of variances

In [ ]:
def rule_of_thumb_ANOVA(dictionary, index):  
    ratio_mag=np.empty((len(dictionary.values())))
    ratio_ph=np.empty((len(dictionary.values())))
    ratio_ex=np.empty((len(dictionary.values())))
    for val,f in zip (dictionary.values(),range(len(dictionary.values()))):
            min_mag=min(val[r"$U_{M},$ [m s^-2/m s^-2]"].values/2)
            max_mag=max(val[r"$U_{M},$ [m s^-2/m s^-2]"].values/2)
            ratio_mag[f]=max_mag/min_mag

            min_ph=min(val[r"$U_{\phi},$ [rad]"].values/2)
            max_ph=max(val[r"$U_{\phi},$ [rad]"].values/2)
            ratio_ph[f]=max_ph/min_ph

            min_ex=min(val[r"$U_{Aexcit},$ [m s^-2/m s^-2]"].values/2)
            max_ex=max(val[r"$U_{Aexcit},$ [m s^-2/m s^-2]"].values/2)
            ratio_ex[f]=max_ex/min_ex
    
    ratios = {'Magnitude' : pd.Series(ratio_mag,index =index),
              'Phase' : pd.Series(ratio_ph,index =index),
              'Excitation amplitude' : pd.Series(ratio_ex,index =index),     
             }
    ratios=pd.DataFrame(ratios, index=index)  
    return ratios#.style.applymap(lambda x: 'background-color : red' if x>2 else 'background-color : green')
 #red - variances are not equal.The assumption might be violated
# green - variances are equal. The assumption is not violated

In [ ]:
list_of_freq=[10,12.5,16,20,25,31.5,40,46.7,50,53.3,63,80,100,125,160,200,250]
rule_PTB=rule_of_thumb_ANOVA(PTB_separated_by_freq,list_of_freq)
rule_CEM=rule_of_thumb_ANOVA(CEM_separated_by_freq_new,list_of_freq)

#create output widgets - optionally
widget1 = widgets.Output()
widget2 = widgets.Output()


with widget1:
    display.display(rule_PTB.style.set_caption('PTB').applymap(lambda x: 'background-color : red' if x>2 else 'background-color : green'))

with widget2:
    display.display(rule_CEM.style.set_caption('CEM').applymap(lambda x: 'background-color : red' if x>2 else 'background-color : green'))



# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2], layout=box_layout)

#render hbox
hbox

The test for the equality of variances shows that distributions do not have the same variance at most frequencies  for both laboratories. Phase observation for both laboratories supports the hypothesis about equality of variances at frequencies up to 25.0 Hz.

This test raise awareness in potential violations in the results which will be provided further by the ANOVA method.


If it is assumed that each experiment is based on at least 30 single values, then the normal distribution can be proposed:
$$X \hookrightarrow  \mathcal{N}(x_{M},\,\sigma _{M}^{2})$$
where $\sigma _{M}$ is calculated as   $\frac{U _{M}}{2}$.

The same approach refers to the phase values: 
$$X \hookrightarrow  \mathcal{N}(x_{\phi},\,\sigma _{\phi}^{2})$$
where $\sigma _{\phi}$ is calculated as   $\frac{U _{\phi}}{2}$.
Now, the task of ANOVA for the magnitude values is to set the null- and research hypothesis and to examine if the null-hypothesis can be rejected or not is to examine whether $x_{M1} = x_{M2}=.....x_{Mi}$
<br> H0:  $x_{M1} = x_{M2}=.....x_{Mi}$
<br>H1 (a research hypothesis): at least one $x_{Mi}$ is different from the rest.

The task of ANOVA for the phase  values is to set the null- and research hypothesis and to examine if the null-hypothesis can be rejected or not is to examine whether $x_{\phi 1} = x_{\phi 2}=.....x_{\phi i}$
<br> H0:  $x_{\phi 1} = x_{\phi 2}=.....x_{\phi i}$
<br>H1 (a research hypothesis): at least one $x_{\phi i}$ is different from the rest.

In both cases, *i* refers to the number of experiments at a given frequency (*i*=10 for PTB and *i*=9 for CEM).

<br> If the p-value of the ANOVA calculation is higher that 0.05, the null hypothesis cannot be rejected. [4] If the p-value is less than 0.05, we reject the null hypothesis that there's no difference between the means.
<br>*NOTE: ANOVA method at this step is based on sampling because for each experiment only the mean value and standard deviation of quantities of interest are known, while the number of single values contained in the each experiment is not known. However, it is not sure whether the sampling is an applicable option for measurement procedure by the acceleration sensor.It is also noticable that the results are affected by the number of samples involved.*

In [ ]:
def ANOVA_through_experiments(dictionary,index,lab):
 
    p=np.empty((len(dictionary.values())))

    for val,f in zip (dictionary.values(),range(len(dictionary.values()))):
        group=[None]*val.shape[0]
        for item in range(val.shape[0]):
            random1=np.random.normal(val[r"$x_{M},$ [m s^-2/m s^-2]"].values[item], val[r"$U_{M},$ [m s^-2/m s^-2]"].values[item]/2, 30)
            group[item]=random1
            
        if lab=="PTB": 
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8],group[9]) 
        elif lab=="CEM":
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8])
        p[f]=pVal
     
    df=pd.DataFrame(p,columns=["p-value (Magnitude) "], index=index) 

          
    for val,f in zip (dictionary.values(),range(len(dictionary.values()))):
        group=[None]*val.shape[0]
        for item in range(val.shape[0]):
            random1=np.random.normal(val[r"$x_{\phi},$ [rad]"].values[item], val[r"$U_{\phi},$ [rad]"].values[item]/2, 30)
            group[item]=random1
        if lab=="PTB": 
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8],group[9])
            p[f]=pVal
        elif lab=="CEM":
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8])
        p[f]=pVal
  
    df["p-value (Phase) "]=p    
        
    return df#.style.applymap(lambda x: 'background-color : yellow' if x<0.05 else 'background-color : green')

 #yellow - the null hypothesis can be rejected. 
# green - the null hypothesis cannot be rejected. 

In [ ]:
p_ANOVA_PTB,p_ANOVA_CEM=ANOVA_through_experiments(PTB_separated_by_freq,list_of_freq,"PTB"),ANOVA_through_experiments(CEM_separated_by_freq_new,list_of_freq,"CEM")

In [ ]:
# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()

with widget1:
    display.display(p_ANOVA_PTB.style.set_caption('PTB').applymap(lambda x: 'background-color : yellow' if x<0.05 else 'background-color : green'))

with widget2:
    display.display(p_ANOVA_CEM.style.set_caption('CEM').applymap(lambda x: 'background-color : yellow' if x<0.05 else 'background-color : green'))



# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2], layout=box_layout)

# render hbox
hbox

# References

[1] https://www.investopedia.com/terms/c/coefficientofvariation.asp
<br>[2] https://en.wikipedia.org/wiki/Weighted_arithmetic_mean
<br>[3] https://en.wikipedia.org/wiki/Effective_sample_size
<br>[4] https://online.stat.psu.edu/stat500/lesson/10/10.2/10.2.1
<br>[5] https://www.marsja.se/levenes-bartletts-test-of-equality-homogeneity-of-variance-in-python/